In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression

In [3]:
spark = SparkSession.builder.appName('LR_Solution').getOrCreate()

21/08/01 14:46:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/08/01 14:46:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/08/01 14:46:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
spark

In [14]:
mushroom_data = spark.read.csv("mushrooms.csv", inferSchema=True, header=True)

In [15]:
mushroom_data.show(5)

+------+--------+----------+--------+-------+------+--------------+-----------+--------+---------+----------+---------+---------------------+---------------------+-------------------+-------------------+--------+---------+----------+--------+---------------+----------+-------+
|   Lab|CapShape|CapSurface|CapColor|Bruises|  Odor|GillAttachment|GillSpacing|GillSize|GillColor|StalkShape|StalkRoot|StalkSurfaceAboveRing|StalkSurfaceBelowRing|StalkColorAboveRing|StalkColorBelowRing|VeilType|VeilColor|RingNumber|RingType|SporePrintColor|Population|Habitat|
+------+--------+----------+--------+-------+------+--------------+-----------+--------+---------+----------+---------+---------------------+---------------------+-------------------+-------------------+--------+---------+----------+--------+---------------+----------+-------+
|EDIBLE|  CONVEX|    SMOOTH|   WHITE|BRUISES|ALMOND|          FREE|    CROWDED|  NARROW|    WHITE|  TAPERING|  BULBOUS|               SMOOTH|               SMOOTH|   

In [18]:
mushroom_data = mushroom_data.drop("VeilType")

In [19]:
features_vector = RFormula(formula= "Lab ~ .")

In [20]:
preprocessed_data = features_vector.fit(mushroom_data).transform(mushroom_data)

In [21]:
preprocessed_data.select("features", "label").show(5, truncate=False)

21/08/01 14:50:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                            |label|
+------------------------------------------------------------------------------------------------------------------------------------+-----+
|(95,[0,6,12,22,26,31,40,41,45,48,51,59,67,70,72,83,84,89],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0  |
|(95,[0,6,12,22,26,31,40,41,45,48,51,59,67,70,72,77,84,89],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0  |
|(95,[0,6,12,22,26,30,40,41,45,48,51,59,67,70,72,83,84,89],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0  |
|(95,[0,6,12,22,26,30,40,41,45,48,51,59,67,70,72,77,84,89],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0  |
|(95,[0,6,12,

In [23]:
train_data, test_data = preprocessed_data.randomSplit([0.7, 0.3])

In [24]:
lr = LogisticRegression(labelCol="label", featuresCol="features")

In [26]:
fitted_lr = lr.fit(train_data)

21/08/01 14:51:34 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/08/01 14:51:34 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [32]:
result = fitted_lr.transform(preprocessed_data)

In [28]:
result.show(5)

+------+--------+----------+--------+-------+----+--------------+-----------+--------+---------+----------+---------+---------------------+---------------------+-------------------+-------------------+---------+----------+--------+---------------+----------+-------+--------------------+-----+--------------------+--------------------+----------+
|   Lab|CapShape|CapSurface|CapColor|Bruises|Odor|GillAttachment|GillSpacing|GillSize|GillColor|StalkShape|StalkRoot|StalkSurfaceAboveRing|StalkSurfaceBelowRing|StalkColorAboveRing|StalkColorBelowRing|VeilColor|RingNumber|RingType|SporePrintColor|Population|Habitat|            features|label|       rawPrediction|         probability|prediction|
+------+--------+----------+--------+-------+----+--------------+-----------+--------+---------+----------+---------+---------------------+---------------------+-------------------+-------------------+---------+----------+--------+---------------+----------+-------+--------------------+-----+-------------

In [33]:
result = result.select("Lab", "prediction")

truePositive = result.filter("prediction = 1.0 AND label = 1.0").count()
print("True Postive: " + str(truePositive))

falsePositive = result.filter("prediction = 1.0 AND label = 0.0").count()
print("False Postive: " + str(falsePositive))

print("")

trueNegative = result.filter("prediction = 0.0 AND label = 0.0").count()
print("True Negative: " + str(trueNegative))

falseNegative = result.filter("prediction = 0.0 AND label = 1.0").count()
print("False Negative: " + str(falseNegative))

True Postive: 3928
False Postive: 0

True Negative: 4488
False Negative: 0


In [23]:
spark.stop()